In [36]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
import shutil

# Paths
INPUT_FOLDER = "Dataset/all_images/"  # Folder where 114 images are stored
OUTPUT_FOLDER = "Classified_Infrared_Dataset"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Create class folders
normal_path = os.path.join(OUTPUT_FOLDER, "Normal")
hot_path = os.path.join(OUTPUT_FOLDER, "Overheated")
os.makedirs(normal_path, exist_ok=True)
os.makedirs(hot_path, exist_ok=True)

# Read images and compute average heat intensity
image_features = []
image_paths = []
for img_file in os.listdir(INPUT_FOLDER):
    img_path = os.path.join(INPUT_FOLDER, img_file)
    img = cv2.imread(img_path)
    img_resized = cv2.resize(img, (100, 100))  # Resize for faster processing
    avg_heat = np.mean(img_resized)  # Compute average pixel intensity

    image_features.append([avg_heat])
    image_paths.append(img_path)

# Apply K-Means Clustering (2 clusters: Normal & Overheated)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(image_features)
labels = kmeans.labels_

# Move images to respective folders based on cluster labels
for i, img_path in enumerate(image_paths):
    if labels[i] == 0:
        shutil.move(img_path, os.path.join(normal_path, os.path.basename(img_path)))
    else:
        shutil.move(img_path, os.path.join(hot_path, os.path.basename(img_path)))

print("✅ Images successfully classified into 'Normal' and 'Overheated' folders!")


✅ Images successfully classified into 'Normal' and 'Overheated' folders!


In [1]:
import cv2
import numpy as np
import os

# Image Preprocessing Function
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    img = cv2.resize(img, (224, 224))  # Resize to match model input size
    img = cv2.GaussianBlur(img, (5, 5), 0)  # Apply Gaussian blur to reduce noise
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)  # Contrast enhancement
    return img / 255.0  # Normalize pixel values to 0-1


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=25,  # More rotation to introduce variety
    width_shift_range=0.2,  # Slightly increase width shifting
    height_shift_range=0.2,  # Slightly increase height shifting
    brightness_range=[0.6, 1.4],  # Simulating more extreme heat conditions
    shear_range=0.2,  # Apply shearing transformation
    zoom_range=0.3,  # Random zoom to introduce scale variation
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True,  # Flip images vertically to simulate different angles
    fill_mode='nearest',  # Fill gaps created by transformations
    validation_split=0.2  # 80% Training, 20% Validation
)

train_generator = datagen.flow_from_directory("classified_infrared_Dataset/", target_size=(224, 224),
                                              batch_size=32, class_mode='binary', subset='training')

val_generator = datagen.flow_from_directory("classified_infrared_Dataset/", target_size=(224, 224),
                                            batch_size=32, class_mode='binary', subset='validation')


Found 92 images belonging to 2 classes.


Found 22 images belonging to 2 classes.


In [9]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load Pretrained Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification: Normal / Overheating

# Define Final Model
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save Model
model.save("battery_heat_model.h5")


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 282s 17us/step


C:\Users\SANGEETH KUMAR L\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.6900 - loss: 0.5423 - val_accuracy: 1.0000 - val_loss: 0.1518
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9760 - loss: 0.1060 - val_accuracy: 0.9091 - val_loss: 0.1167
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 1.0000 - loss: 0.0166 - val_accuracy: 0.9091 - val_loss: 0.3158
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9946 - loss: 0.0705 - val_accuracy: 0.8636 - val_loss: 0.7188
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9730 - loss: 0.0952 - val_accuracy: 0.8182 - val_loss: 0.5174
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9294 - loss: 0.1953 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9904 - loss: 0.0232 - val_accuracy: 0.9545 - val_loss: 0.2098
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9815 - loss: 0.0355 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 9/10
3/3 

In [56]:
pip install flask pymongo gridfs pillow tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gridfs (from versions: none)
ERROR: No matching distribution found for gridfs
You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from flask import Flask, request, render_template, jsonify
from flask_cors import CORS
import base64
import io
from PIL import Image
import numpy as np
import tensorflow as tf
from pymongo import MongoClient
from bson import ObjectId
import asyncio  # Import asyncio for async functions

# Import heat control functions
from battery_control import get_latest_battery_data, adjust_battery_power

app = Flask(__name__)
CORS(app)

# ✅ Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["battery_monitoring"]
collection = db["predictions"]

# ✅ Load AI Model
MODEL_PATH = "battery_heat_model.h5"
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model Loaded Successfully")

def preprocess_image(image: Image.Image):
    """Preprocess the image before feeding it into the model."""
    image = image.resize((224, 224))  # Resize to match model input size
    image = np.array(image) / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    try:
        file = request.files['file']
        if not file:
            return jsonify({'error': 'No file uploaded'}), 400

        if not file.content_type in ['image/jpeg', 'image/jpg']:
            return jsonify({'error': 'Invalid file type. Please upload a JPG/JPEG image.'}), 400

        # ✅ Convert Image to Base64
        image = Image.open(file.stream)
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

        # ✅ Process Image for AI Model
        processed_image = preprocess_image(image)
        prediction = model.predict(processed_image)
        heat_issue_prob = float(prediction[0][0])  # Extract probability
        status = "Battery Overheating!" if heat_issue_prob > 0.5 else "Battery Temperature Normal."

        # ✅ Store Everything in MongoDB
        result = {
            "image_base64": img_base64,
            "image_name": file.filename,
            "heat_issue_probability": heat_issue_prob,
            "status": status
        }
        insert_result = collection.insert_one(result)

        # Convert ObjectId to string
        result['_id'] = str(insert_result.inserted_id)

        return jsonify(result)
    except Exception as e:
        print("Exception occurred:", str(e))
        return jsonify({"error": str(e)}), 500

@app.route('/heat_controller', methods=['GET'])
def heat_controller():
    """Fetch latest battery data and decide cooling adjustments."""
    try:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        control_data = loop.run_until_complete(adjust_battery_power())  # Run async function
        return jsonify(control_data)
    except Exception as e:
        print("Error in heat control:", str(e))
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, host="127.0.0.1", port=5000, use_reloader=False)


✅ Model Loaded Successfully
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:39:26] "GET /battery-status HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:39:26] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:28] "GET /heat_controller HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:28] "GET /heat_controller HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:28] "GET /heat_controller HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:28] "GET /heat_controller HTTP/1.1" 500 -


Error in heat control: Task <Task pending name='Task-94' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-95' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-96' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop


INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:34] "GET /heat_controller HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:34] "GET /heat_controller HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:34] "GET /heat_controller HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:43:34] "GET /heat_controller HTTP/1.1" 500 -


Error in heat control: Task <Task pending name='Task-97' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-98' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-99' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-100' coro=<adjust_battery_power() running at e:\Codevolt\batte

...

Error in heat control: Task <Task pending name='Task-101' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-102' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-104' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-103' coro=<adjust_battery_power() running at e:\Codevolt\ba

INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:45:32] "GET /heat_controller HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:45:32] "GET /heat_controller HTTP/1.1" 500 -


Error in heat control: Task <Task pending name='Task-106' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-107' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop


INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:45:41] "GET /heat_controller HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 07:45:41] "GET /heat_controller HTTP/1.1" 500 -


Error in heat control: Task <Task pending name='Task-108' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
Error in heat control: Task <Task pending name='Task-109' coro=<adjust_battery_power() running at e:\Codevolt\battery_control.py:8>> got Future <Future pending cb=[_chain_future.<locals>._call_check_cancel() at c:\Program Files\Python310\lib\asyncio\futures.py:384]> attached to a different loop
